In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True )
import pandas as pd
#%cd /content/drive/My Drive/bases_mentoria/
#path= '/content/drive/My Drive/bases_mentoria/primer trimestre 2019'
path= '/content/drive/My Drive/Mentoria/primer trimestre 2019'
import glob
filenames = glob.glob(path + "/*.csv")

dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename, encoding='latin-1', sep=";"))

# Concatenate all data into one DataFrame
big_frame = pd.concat(dfs, ignore_index=True)

big_frame.head()


Mounted at /content/drive/


,FECHA,FECHAAPERTURA,TARJETA,CORREDOR,LINEA,SENTIDO
0,01/01/2019 01:32:43 AM,01/01/2019 01:00:11 AM,4992639,Cor 3 Rojo,L35,Ida
1,01/01/2019 02:26:11 AM,01/01/2019 01:00:11 AM,5489022,Cor 3 Rojo,L35,Vuelta
2,01/01/2019 02:26:14 AM,01/01/2019 01:00:11 AM,5489022,Cor 3 Rojo,L35,Vuelta
3,01/01/2019 02:32:27 AM,01/01/2019 01:59:31 AM,5317539,Cor 1 Naranja,L12,Vuelta
4,01/01/2019 02:57:48 AM,01/01/2019 01:59:31 AM,4451141,Cor 1 Naranja,L12,Vuelta


In [ ]:
big_frame['m'] = big_frame.FECHA.astype(str).str[:10].astype(str)

In [ ]:
big_frame['hora'] = big_frame.FECHA.astype(str).str[11:].astype(str)

In [ ]:
big_frame['aux']=big_frame.FECHA.astype(str).str[19:].astype(str) 


In [ ]:
big_frame['aux2']=big_frame.hora.astype(str).str[:2].astype(str) 

In [ ]:
big_frame['hh']=big_frame.aux2 + big_frame.aux

In [ ]:
big_frame.sort_values(by=['hh'], inplace=True)

In [ ]:
big_frame.m.value_counts()

28/03/2019    176887
29/03/2019    176481
13/03/2019    174942
12/03/2019    173855
11/03/2019    173682
               ...  
20/01/2019     34871
27/01/2019     34737
13/01/2019     29166
06/01/2019     22635
01/01/2019     13487
Name: m, Length: 90, dtype: int64

In [ ]:
big_frame['dia'] = pd.to_datetime(big_frame['m'])
big_frame['dia'] = big_frame['dia'].dt.strftime('%d.%m.%Y')

In [ ]:
big_frame.dia.value_counts()

28.03.2019    176887
29.03.2019    176481
13.03.2019    174942
03.12.2019    173855
03.11.2019    173682
               ...  
20.01.2019     34871
27.01.2019     34737
13.01.2019     29166
01.06.2019     22635
01.01.2019     13487
Name: dia, Length: 90, dtype: int64

In [ ]:
big_frame['dia2'] = pd.to_datetime(big_frame['dia'])

In [ ]:

### EJERCICIO 1: INTERACTUAR CON BASES, CONVERTIR FORMATOS FECHAS, GENERAR VARIABLES NUEVAS (MES DIA )###
big_frame['day_of_week'] = big_frame['dia2'].dt.day_name()


In [ ]:
big_frame.dia2.value_counts()

2019-03-28    176887
2019-03-29    176481
2019-03-13    174942
2019-03-12    173855
2019-03-11    173682
               ...  
2019-01-20     34871
2019-01-27     34737
2019-01-13     29166
2019-01-06     22635
2019-01-01     13487
Name: dia2, Length: 90, dtype: int64

In [ ]:
big_frame.shape

(10290050, 14)

In [ ]:
###EJERCICIO 2: 
##APLICAR ESTADÍSTICOS DESCRIPTIVOS SOBRE CORTES DE BOLETOS: 
## 1) VARIABLES ESPACIALES: CORREDOR, LINEA Y SENTIDO
  ## CONTAMOS VIAJES POR  POR CORREDOR, LINEA Y SENTIDO , POR DIA###
big_frame['AUX']=1
big_frame['VIAJES_DIA_HORA_CORREDOR']=big_frame.groupby(['CORREDOR', 'dia2', 'hh'])['AUX'].transform('sum')
big_frame['VIAJES_DIA_HORA_LINEA']=big_frame.groupby(['LINEA', 'dia2', 'hh'])['AUX'].transform('sum')
big_frame['SENTIDO_IDA_HORA']=0
mask=big_frame.SENTIDO=="Ida"
big_frame.loc[mask,'SENTIDO_IDA_HORA']=1
big_frame['SENTIDO_VUELTA_HORA']=0
mask=big_frame.SENTIDO=="Vuelta"
big_frame.loc[mask,'SENTIDO_VUELTA_HORA']=1

big_frame['VIAJES_DIA_HORA_LINEA_SENT_IDA']=big_frame.groupby(['LINEA', 'dia2', 'hh'])['SENTIDO_IDA_HORA'].transform('sum')
big_frame['VIAJES_DIA_HORA_LINEA_SENT_VUELTA']=big_frame.groupby(['LINEA', 'dia2', 'hh'])['SENTIDO_VUELTA_HORA'].transform('sum')
big_frame.head(10)

big_frame= big_frame.drop_duplicates(['CORREDOR', 'LINEA', 'dia2', 'VIAJES_DIA_HORA_CORREDOR', 	'VIAJES_DIA_HORA_LINEA' , 'VIAJES_DIA_HORA_LINEA_SENT_IDA', 	'VIAJES_DIA_HORA_LINEA_SENT_VUELTA', 'hh' ])
big_frame.head()
big_frame=big_frame.loc[:,['CORREDOR', 'LINEA', 'dia2',  'VIAJES_DIA_HORA_CORREDOR', 	'VIAJES_DIA_HORA_LINEA' , 'VIAJES_DIA_HORA_LINEA_SENT_IDA', 	'VIAJES_DIA_HORA_LINEA_SENT_VUELTA', 'hh' ]]
#df1.to_csv("df1.csv", index=None)
#from google.colab import files
#files.download('df1.csv') 

## 2) VARIABLES TEMPORALES: MES DIA DE LA SEMANA Y HORA##
## 3) ANALIZAR ESTACIONALIDAD
## IDEM SOBRE TARJETAS MESES USADAS VS DIAS PROMEDIO USADAS (¿OUTLIERS?, CANTIDAD DE USOS DIA)

###LUEGO GUARDAMOS UN DF PARA PODER LUEGO HACER LOOP, DIGAMOS, DEBERIA HABER 2 LOOP, EL QUE LEVANTA Y PROCESA Y LUEGO EL QEU UNE TODO##

In [ ]:
big_frame.shape

(126191, 8)

In [ ]:
viajes_1t19= big_frame
viajes_1t19.to_csv("viajes_hora_1t19.csv", index=None)
from google.colab import files
files.download('viajes_hora_1t19.csv') 



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
##LO MISMO PARA EL SEGUNDO TRIMESTRE DE 2019#
from google.colab import drive
drive.mount('/content/drive/', force_remount=True )
import pandas as pd
#%cd /content/drive/My Drive/bases_mentoria/
path= '/content/drive/My Drive/Mentoria/segundo trimestre 2019'
import glob
filenames = glob.glob(path + "/*.csv")

dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename, encoding='latin-1', sep=";"))

# Concatenate all data into one DataFrame
big_frame = pd.concat(dfs, ignore_index=True)

big_frame.head()




Mounted at /content/drive/


,FECHA,FECHAAPERTURA,TARJETA,CORREDOR,LINEA,SENTIDO
0,05/04/2019 12:00:08 AM,04/04/2019 10:02:49 PM,4884082,Cor 8 Rojo,L82,Vuelta
1,05/04/2019 12:00:10 AM,04/04/2019 11:30:03 PM,5871414,TROLE,B,Vuelta
2,05/04/2019 12:00:12 AM,04/04/2019 10:43:21 PM,5813430,Cor 7 Azul,L75,Vuelta
3,05/04/2019 12:00:13 AM,04/04/2019 11:42:13 PM,3262031,Cor 2 Azul,L26,Ida
4,05/04/2019 12:00:15 AM,04/04/2019 10:59:19 PM,5791090,Cor 7 Azul,L74,Vuelta


In [ ]:
big_frame['m'] = big_frame.FECHA.astype(str).str[:10].astype(str)

In [ ]:
big_frame['hora'] = big_frame.FECHA.astype(str).str[11:].astype(str)

In [ ]:
big_frame['aux']=big_frame.FECHA.astype(str).str[19:].astype(str) 

In [ ]:
big_frame['aux2']=big_frame.hora.astype(str).str[:2].astype(str) 

In [ ]:
big_frame['hh']=big_frame.aux2 + big_frame.aux

In [ ]:
big_frame.sort_values(by=['hh'], inplace=True)

In [ ]:
big_frame.m.value_counts()

10/05/2019    191236
14/06/2019    189752
07/06/2019    188931
13/06/2019    188717
16/05/2019    188680
               ...  
05/05/2019     41274
21/04/2019     39324
16/06/2019     37555
01/05/2019       555
29/05/2019       383
Name: m, Length: 91, dtype: int64

In [ ]:
big_frame['dia'] = pd.to_datetime(big_frame['m'])
big_frame['dia'] = big_frame['dia'].dt.strftime('%d.%m.%Y')

In [ ]:
big_frame.dia.value_counts()

05.10.2019    191236
14.06.2019    189752
06.07.2019    188931
13.06.2019    188717
16.05.2019    188680
               ...  
05.05.2019     41274
21.04.2019     39324
16.06.2019     37555
05.01.2019       555
29.05.2019       383
Name: dia, Length: 91, dtype: int64

In [ ]:
big_frame['dia2'] = pd.to_datetime(big_frame['dia'])

In [ ]:
### EJERCICIO 1: INTERACTUAR CON BASES, CONVERTIR FORMATOS FECHAS, GENERAR VARIABLES NUEVAS (MES DIA )###


In [ ]:
big_frame.shape

(12607201, 13)

In [ ]:
###EJERCICIO 2: 
##APLICAR ESTADÍSTICOS DESCRIPTIVOS SOBRE CORTES DE BOLETOS: 
## 1) VARIABLES ESPACIALES: CORREDOR, LINEA Y SENTIDO
  ## CONTAMOS VIAJES POR  POR CORREDOR, LINEA Y SENTIDO , POR DIA###
big_frame['AUX']=1
big_frame['VIAJES_DIA_HORA_CORREDOR']=big_frame.groupby(['CORREDOR', 'dia2', 'hh'])['AUX'].transform('sum')
big_frame['VIAJES_DIA_HORA_LINEA']=big_frame.groupby(['LINEA', 'dia2', 'hh'])['AUX'].transform('sum')
big_frame['SENTIDO_IDA_HORA']=0
mask=big_frame.SENTIDO=="Ida"
big_frame.loc[mask,'SENTIDO_IDA_HORA']=1
big_frame['SENTIDO_VUELTA_HORA']=0
mask=big_frame.SENTIDO=="Vuelta"
big_frame.loc[mask,'SENTIDO_VUELTA_HORA']=1

big_frame['VIAJES_DIA_HORA_LINEA_SENT_IDA']=big_frame.groupby(['LINEA', 'dia2', 'hh'])['SENTIDO_IDA_HORA'].transform('sum')
big_frame['VIAJES_DIA_HORA_LINEA_SENT_VUELTA']=big_frame.groupby(['LINEA', 'dia2', 'hh'])['SENTIDO_VUELTA_HORA'].transform('sum')
big_frame.head(10)

big_frame= big_frame.drop_duplicates(['CORREDOR', 'LINEA', 'dia2', 'VIAJES_DIA_HORA_CORREDOR', 	'VIAJES_DIA_HORA_LINEA' , 'VIAJES_DIA_HORA_LINEA_SENT_IDA', 	'VIAJES_DIA_HORA_LINEA_SENT_VUELTA', 'hh' ])
big_frame.head()
big_frame=big_frame.loc[:,['CORREDOR', 'LINEA', 'dia2',  'VIAJES_DIA_HORA_CORREDOR', 	'VIAJES_DIA_HORA_LINEA' , 'VIAJES_DIA_HORA_LINEA_SENT_IDA', 	'VIAJES_DIA_HORA_LINEA_SENT_VUELTA', 'hh' ]]
#df1.to_csv("df1.csv", index=None)
#from google.colab import files
#files.download('df1.csv') 

## 2) VARIABLES TEMPORALES: MES DIA DE LA SEMANA Y HORA##
## 3) ANALIZAR ESTACIONALIDAD
## IDEM SOBRE TARJETAS MESES USADAS VS DIAS PROMEDIO USADAS (¿OUTLIERS?, CANTIDAD DE USOS DIA)

###LUEGO GUARDAMOS UN DF PARA PODER LUEGO HACER LOOP, DIGAMOS, DEBERIA HABER 2 LOOP, EL QUE LEVANTA Y PROCESA Y LUEGO EL QEU UNE TODO##

In [ ]:
big_frame.shape

In [ ]:
big_frame.dia2.value_counts()

In [ ]:
viajes_2t19= big_frame
viajes_2t19.to_csv("viajes_hora_2t19.csv", index=None)
from google.colab import files
files.download('viajes_hora_2t19.csv') 


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
##LO MISMO PARA EL TERCER TRIMESTRE DE 2019#
from google.colab import drive
drive.mount('/content/drive/', force_remount=True )
import pandas as pd
#%cd /content/drive/My Drive/bases_mentoria/
path= '/content/drive/My Drive/Mentoria/tercer trimestre 2019'
import glob
filenames = glob.glob(path + "/*.csv")

dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename, encoding='latin-1', sep=";"))

# Concatenate all data into one DataFrame
big_frame = pd.concat(dfs, ignore_index=True)

big_frame.head()




Mounted at /content/drive/


,FECHA,FECHAAPERTURA,TARJETA,CORREDOR,LINEA,SENTIDO
0,06/07/2019 12:00:07 AM,05/07/2019 11:28:07 PM,5099043,Cor 1 Naranja,L12,Ida
1,06/07/2019 12:00:29 AM,05/07/2019 11:32:24 PM,5515754,Cor 6 Naranja,L67,Ida
2,06/07/2019 12:00:29 AM,05/07/2019 11:05:36 PM,4971142,Cor 5 Amarillo,L50,Vuelta
3,06/07/2019 12:00:30 AM,05/07/2019 11:05:36 PM,4971142,Cor 5 Amarillo,L50,Vuelta
4,06/07/2019 12:00:33 AM,05/07/2019 11:05:36 PM,4971142,Cor 5 Amarillo,L50,Vuelta


In [ ]:
big_frame['m'] = big_frame.FECHA.astype(str).str[:10].astype(str)

In [ ]:
big_frame['hora'] = big_frame.FECHA.astype(str).str[11:].astype(str)

In [ ]:
big_frame['aux']=big_frame.FECHA.astype(str).str[19:].astype(str) 

In [ ]:
big_frame['aux2']=big_frame.hora.astype(str).str[:2].astype(str) 

In [ ]:
big_frame['hh']=big_frame.aux2 + big_frame.aux

In [ ]:
big_frame.sort_values(by=['hh'], inplace=True)

In [ ]:
big_frame.m.value_counts()

In [ ]:
big_frame['dia'] = pd.to_datetime(big_frame['m'])
big_frame['dia'] = big_frame['dia'].dt.strftime('%d.%m.%Y')

In [ ]:
big_frame.dia.value_counts()

In [ ]:
big_frame['dia2'] = pd.to_datetime(big_frame['dia'])

In [ ]:
### EJERCICIO 1: INTERACTUAR CON BASES, CONVERTIR FORMATOS FECHAS, GENERAR VARIABLES NUEVAS (MES DIA )###
big_frame['day_of_week'] = big_frame['dia2'].dt.day_name()


In [ ]:
big_frame.dia2.value_counts()

In [ ]:
big_frame.shape

(12718216, 13)

In [ ]:
###EJERCICIO 2: 
##APLICAR ESTADÍSTICOS DESCRIPTIVOS SOBRE CORTES DE BOLETOS: 
## 1) VARIABLES ESPACIALES: CORREDOR, LINEA Y SENTIDO
  ## CONTAMOS VIAJES POR  POR CORREDOR, LINEA Y SENTIDO , POR DIA###
big_frame['AUX']=1
big_frame['VIAJES_DIA_HORA_CORREDOR']=big_frame.groupby(['CORREDOR', 'dia2', 'hh'])['AUX'].transform('sum')
big_frame['VIAJES_DIA_HORA_LINEA']=big_frame.groupby(['LINEA', 'dia2', 'hh'])['AUX'].transform('sum')
big_frame['SENTIDO_IDA_HORA']=0
mask=big_frame.SENTIDO=="Ida"
big_frame.loc[mask,'SENTIDO_IDA_HORA']=1
big_frame['SENTIDO_VUELTA_HORA']=0
mask=big_frame.SENTIDO=="Vuelta"
big_frame.loc[mask,'SENTIDO_VUELTA_HORA']=1

big_frame['VIAJES_DIA_HORA_LINEA_SENT_IDA']=big_frame.groupby(['LINEA', 'dia2', 'hh'])['SENTIDO_IDA_HORA'].transform('sum')
big_frame['VIAJES_DIA_HORA_LINEA_SENT_VUELTA']=big_frame.groupby(['LINEA', 'dia2', 'hh'])['SENTIDO_VUELTA_HORA'].transform('sum')
big_frame.head(10)

big_frame= big_frame.drop_duplicates(['CORREDOR', 'LINEA', 'dia2', 'VIAJES_DIA_HORA_CORREDOR', 	'VIAJES_DIA_HORA_LINEA' , 'VIAJES_DIA_HORA_LINEA_SENT_IDA', 	'VIAJES_DIA_HORA_LINEA_SENT_VUELTA', 'hh' ])
big_frame.head()
big_frame=big_frame.loc[:,['CORREDOR', 'LINEA', 'dia2',  'VIAJES_DIA_HORA_CORREDOR', 	'VIAJES_DIA_HORA_LINEA' , 'VIAJES_DIA_HORA_LINEA_SENT_IDA', 	'VIAJES_DIA_HORA_LINEA_SENT_VUELTA', 'hh' ]]
#df1.to_csv("df1.csv", index=None)
#from google.colab import files
#files.download('df1.csv') 

## 2) VARIABLES TEMPORALES: MES DIA DE LA SEMANA Y HORA##
## 3) ANALIZAR ESTACIONALIDAD
## IDEM SOBRE TARJETAS MESES USADAS VS DIAS PROMEDIO USADAS (¿OUTLIERS?, CANTIDAD DE USOS DIA)

###LUEGO GUARDAMOS UN DF PARA PODER LUEGO HACER LOOP, DIGAMOS, DEBERIA HABER 2 LOOP, EL QUE LEVANTA Y PROCESA Y LUEGO EL QEU UNE TODO##

In [ ]:
big_frame.shape

In [ ]:
big_frame.dia2.value_counts()

In [ ]:
viajes_3t19= big_frame
viajes_3t19.to_csv("viajes_hora_3t19.csv", index=None)
from google.colab import files
files.download('viajes_hora_3t19.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
##LO MISMO PARA EL CUARTO TRIMESTRE DE 2019#
from google.colab import drive
drive.mount('/content/drive/', force_remount=True )
import pandas as pd
#%cd /content/drive/My Drive/bases_mentoria/
path= '/content/drive/My Drive/Mentoria/cuarto trimestre 2019'
import glob
filenames = glob.glob(path + "/*.csv")

dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename, encoding='latin-1', sep=";"))

# Concatenate all data into one DataFrame
big_frame = pd.concat(dfs, ignore_index=True)

big_frame.head()

Mounted at /content/drive/


,FECHA,FECHAAPERTURA,TARJETA,CORREDOR,LINEA,SENTIDO
0,06/10/2019 12:00:05 AM,05/10/2019 11:02:36 PM,5969486,Cor 7 Azul,L75,Vuelta
1,06/10/2019 12:00:09 AM,05/10/2019 11:26:36 PM,5943292,Cor 1 Naranja,L13,Ida
2,06/10/2019 12:00:13 AM,05/10/2019 11:46:15 PM,3497303,Cor 3 Rojo,L33,Ida
3,06/10/2019 12:00:15 AM,05/10/2019 11:46:15 PM,4659858,Cor 3 Rojo,L33,Ida
4,06/10/2019 12:00:18 AM,05/10/2019 10:27:52 PM,5325561,Cor 6 Naranja,L60,Vuelta


In [ ]:
big_frame['m'] = big_frame.FECHA.astype(str).str[:10].astype(str)

In [ ]:
big_frame['hora'] = big_frame.FECHA.astype(str).str[11:].astype(str)

In [ ]:
big_frame['aux']=big_frame.FECHA.astype(str).str[19:].astype(str) 

In [ ]:
big_frame['aux2']=big_frame.hora.astype(str).str[:2].astype(str) 

In [ ]:
big_frame['hh']=big_frame.aux2 + big_frame.aux

In [ ]:
big_frame.sort_values(by=['hh'], inplace=True)

In [ ]:
big_frame.m.value_counts()

In [ ]:
big_frame['dia'] = pd.to_datetime(big_frame['m'])
big_frame['dia'] = big_frame['dia'].dt.strftime('%d.%m.%Y')

In [ ]:
big_frame.info()

In [ ]:
big_frame.dia.value_counts()

In [ ]:
big_frame['dia2'] = pd.to_datetime(big_frame['dia'])

In [ ]:
### EJERCICIO 1: INTERACTUAR CON BASES, CONVERTIR FORMATOS FECHAS, GENERAR VARIABLES NUEVAS (MES DIA )###
big_frame['day_of_week'] = big_frame['dia2'].dt.day_name()


In [ ]:
big_frame.dia2.value_counts()

In [ ]:
big_frame.shape

In [ ]:
###EJERCICIO 2: 
##APLICAR ESTADÍSTICOS DESCRIPTIVOS SOBRE CORTES DE BOLETOS: 
## 1) VARIABLES ESPACIALES: CORREDOR, LINEA Y SENTIDO
  ## CONTAMOS VIAJES POR  POR CORREDOR, LINEA Y SENTIDO , POR DIA###
big_frame['AUX']=1
big_frame['VIAJES_DIA_HORA_CORREDOR']=big_frame.groupby(['CORREDOR', 'dia2', 'hh'])['AUX'].transform('sum')
big_frame['VIAJES_DIA_HORA_LINEA']=big_frame.groupby(['LINEA', 'dia2', 'hh'])['AUX'].transform('sum')
big_frame['SENTIDO_IDA_HORA']=0
mask=big_frame.SENTIDO=="Ida"
big_frame.loc[mask,'SENTIDO_IDA_HORA']=1
big_frame['SENTIDO_VUELTA_HORA']=0
mask=big_frame.SENTIDO=="Vuelta"
big_frame.loc[mask,'SENTIDO_VUELTA_HORA']=1

big_frame['VIAJES_DIA_HORA_LINEA_SENT_IDA']=big_frame.groupby(['LINEA', 'dia2', 'hh'])['SENTIDO_IDA_HORA'].transform('sum')
big_frame['VIAJES_DIA_HORA_LINEA_SENT_VUELTA']=big_frame.groupby(['LINEA', 'dia2', 'hh'])['SENTIDO_VUELTA_HORA'].transform('sum')
big_frame.head(10)

big_frame= big_frame.drop_duplicates(['CORREDOR', 'LINEA', 'dia2', 'VIAJES_DIA_HORA_CORREDOR', 	'VIAJES_DIA_HORA_LINEA' , 'VIAJES_DIA_HORA_LINEA_SENT_IDA', 	'VIAJES_DIA_HORA_LINEA_SENT_VUELTA', 'hh' ])
big_frame.head()
big_frame=big_frame.loc[:,['CORREDOR', 'LINEA', 'dia2',  'VIAJES_DIA_HORA_CORREDOR', 	'VIAJES_DIA_HORA_LINEA' , 'VIAJES_DIA_HORA_LINEA_SENT_IDA', 	'VIAJES_DIA_HORA_LINEA_SENT_VUELTA', 'hh' ]]
#df1.to_csv("df1.csv", index=None)
#from google.colab import files
#files.download('df1.csv') 

## 2) VARIABLES TEMPORALES: MES DIA DE LA SEMANA Y HORA##
## 3) ANALIZAR ESTACIONALIDAD
## IDEM SOBRE TARJETAS MESES USADAS VS DIAS PROMEDIO USADAS (¿OUTLIERS?, CANTIDAD DE USOS DIA)

###LUEGO GUARDAMOS UN DF PARA PODER LUEGO HACER LOOP, DIGAMOS, DEBERIA HABER 2 LOOP, EL QUE LEVANTA Y PROCESA Y LUEGO EL QEU UNE TODO##

In [ ]:
big_frame.shape

In [ ]:
big_frame.dia2.value_counts()

In [ ]:
viajes_4t19= big_frame
viajes_4t19.to_csv("viajes_hora_4t19.csv", index=None)
from google.colab import files
files.download('viajes_hora_4t19.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>